In [ ]:
# -*- coding: utf-8 -*-

#標準ライブラリ
import time
import re
import os
import datetime
from xml.etree import ElementTree as etree

#サードパーティ(要インストール)
import pandas as pd
import numpy as np
import lxml.html
import requests as req
from concurrent import futures

In [ ]:
#html型式から変換する関数
def transform_html(url):
    html_data = req.get(url)
    converted_data = lxml.html.fromstring(html_data.content)
    
    return converted_data

In [ ]:
'''ページ数計算
ページ数が不明であるため、サイト内の求人広告総数を１ページあたりの広告数で割ることで算出
'''
def get_last_page(root_url):
    url_sum_xpath = './/span[@class="c-text -highlightNumber"]/text()' #求人広告総数
    page_url_xpath = './/li[@class="p-contentList__jobList__item"]' #１ページあたりの求人広告数
    converted_data = transform_html(root_url)

    page_url = converted_data.findall(page_url_xpath)
    url_sum = converted_data.xpath(url_sum_xpath)
    url_sum = url_sum[0]
    url_sum = url_sum.replace(',','')

    page_num = int(url_sum)//len(page_url)+1 #あまりがでるので小数点以下切り捨てして１ページ分追加


    return page_num

In [ ]:
%%time
'''
このサイトの求人全てを取得するには、地域ごとから取得することによりできる。
'''
ROOT_URL = 'https://04510.jp/jobs/areas'
tiiki_list = ['tohoku','kanto','tokai','kansai','hokuriku','chugoku','kyushu']
url_list = []
url_xpath = './/div[@class="p-cardJob__button"]/a[@class="c-button -detail"]'

for i in range(0,7):
    tiiki = tiiki_list[i]
    tiiki_url = '/'.join([ROOT_URL,tiiki])
    last_page = get_last_page(tiiki_url)
    page_num = 1
    while page_num <= last_page:
        param = '?page={}'.format(page_num)
        combined_url = '/'.join([tiiki_url,param])
        converted_data = transform_html(combined_url)
        try:
            for element in converted_data.findall(url_xpath):
                url_list.append(element.get('href'))
            print(tiiki+':'+str(page_num))
        except:
            print('fail:page '+tiiki)
        page_num +=1
        # time.sleep(1)

url_df = pd.DataFrame(url_list,columns=["url"])
url_df.to_csv('url.csv',index=False)

In [ ]:
url_df = pd.read_csv('url.csv')

In [ ]:
'''
取得したURLから求人広告の概要を取得するための関数
必要ないデータもまとめて取得してあとからいらないものを削除するようにしています
'''
def get_data(i):

    hr_dict = {}
    url = url_df['url'][i]
    converted_data = transform_html(url)
    
    job_requirement_dt_xpath = './/div[@class="p-informationListView"]/dl/dt/h4' #辞書のキーにする部分
    job_requirement_dt = converted_data.findall(job_requirement_dt_xpath)

    job_requirement_dd_xpath = './/div[@class="p-informationListView"]/dl/dd' #辞書のバリューにする部分
    job_requirement_dd = converted_data.findall(job_requirement_dd_xpath)

    for job_requirement_dt, job_requirement_dd in zip(job_requirement_dt, job_requirement_dd):
        job_requirement_dd_text = ''
        job_requirement_dt_text = job_requirement_dt.text
        job_requirement_dd_texts = job_requirement_dd.itertext() #バリューの指定したxpathの子階層のテキストも全取得する
        for dd_text in job_requirement_dd_texts:
            job_requirement_dd_text += dd_text
        hr_dict.setdefault(job_requirement_dt_text, job_requirement_dd_text)

    title = converted_data.xpath('.//h2[@class="c-indexLevel2"]/text()')
    hr_dict.setdefault('ページタイトル',title)


    # get_day = '{}/{}/{}'.format(year, month, day)
    baitai = '???'

    hr_dict.setdefault('ページurl', url)
    # hr_dict.setdefault('取得日', get_day)
    hr_dict.setdefault('媒体名', baitai)
    
    time.sleep(2)

    print(i)

    return hr_dict

In [ ]:
%%time
'''
並行処理で取得していく
joblibも使用したことがあるがこちらのほうが止まることなくスムーズに処理ができた
'''
myThreads = []
with futures.ThreadPoolExecutor(max_workers=24) as executor:
    for i in range(0,30000):
        try: 
            myThreads.append(executor.submit(get_data,i))
        except:
            print('error')
            continue

result = []
for x in myThreads:
    try:
        item = x.result()
        result.append(item)
    except:
        pass

df = pd.DataFrame(result)
df

In [ ]:
df.to_csv('data.csv')